In [ ]:
# UCSD Data Science Bootcamp, HW 21 ML
# Alexis Perumal, 4/28/20
# Update sklearn to prevent version mismatches
#
# Model 3 - Random Forest Classifier
#
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [4]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [5]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [6]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]
selected_features = df[ ['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact','koi_duration', 'koi_depth', 'koi_prad',
       'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
       'koi_srad', 'ra', 'dec', 'koi_kepmag'] ]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
# y = df["koi_disposition"].map({'CANDIDATE':0.0, 'CONFIRMED':1.0, 'FALSE POSITIVE':0.0}).values.reshape(-1, 1)
y = df["koi_disposition"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=1)

y

0            CONFIRMED
1       FALSE POSITIVE
2       FALSE POSITIVE
3            CONFIRMED
4            CONFIRMED
             ...      
6986    FALSE POSITIVE
6987    FALSE POSITIVE
6988         CANDIDATE
6989    FALSE POSITIVE
6990    FALSE POSITIVE
Name: koi_disposition, Length: 6991, dtype: object

In [9]:
print("X_train length: ", len(X_train))
print("X_test length: ", len(X_test))
print("y_train length: ", len(y_train))
print("y_test length: ", len(y_test))
X_train.head()
X_train.shape

X_train length:  5243
X_test length:  1748
y_train length:  5243
y_test length:  1748


(5243, 20)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

In [11]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

# Classifier Model of Choice

In [12]:
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression(random_state=99)
# classifier

In [13]:
# # Note SVM needs sclaed data, but random forest doesn't.
# from sklearn.svm import SVC
# classifier = SVC()
# classifier

In [14]:
# Note SVM needs sclaed data, but random forest doesn't.
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=99)
classifier

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [18]:
y_train = y_train.ravel()

In [19]:
model2 = classifier.fit(X_train_scaled, y_train)

# Train the Model



In [20]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9107551487414187


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [21]:
# # Create the GridSearchCV model for Logistic Regression
# from sklearn.model_selection import GridSearchCV
# param_grid = {'C': [0.1, 0.5, 1, 1.5, 2, 3, 5, 7, 10],
#               'penalty': ["l1", "l2"]}
# grid = GridSearchCV(model2, param_grid, verbose=3)

In [24]:
# Create the GridSearchCV model for Logistic Regression
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [50, 75, 100, 125, 150],
              'max_features':[None, 'auto', 'sqrt']
              }
grid = GridSearchCV(model2, param_grid, verbose=3)

In [25]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] max_features=None, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. max_features=None, n_estimators=50, score=0.888, total=   2.0s
[CV] max_features=None, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] .. max_features=None, n_estimators=50, score=0.886, total=   2.0s
[CV] max_features=None, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s


[CV] .. max_features=None, n_estimators=50, score=0.891, total=   2.0s
[CV] max_features=None, n_estimators=50 ..............................
[CV] .. max_features=None, n_estimators=50, score=0.902, total=   2.0s
[CV] max_features=None, n_estimators=50 ..............................
[CV] .. max_features=None, n_estimators=50, score=0.887, total=   2.0s
[CV] max_features=None, n_estimators=75 ..............................
[CV] .. max_features=None, n_estimators=75, score=0.888, total=   3.1s
[CV] max_features=None, n_estimators=75 ..............................
[CV] .. max_features=None, n_estimators=75, score=0.890, total=   3.0s
[CV] max_features=None, n_estimators=75 ..............................
[CV] .. max_features=None, n_estimators=75, score=0.890, total=   3.0s
[CV] max_features=None, n_estimators=75 ..............................
[CV] .. max_features=None, n_estimators=75, score=0.905, total=   3.0s
[CV] max_features=None, n_estimators=75 ..............................
[CV] .

[CV] . max_features=sqrt, n_estimators=100, score=0.895, total=   1.1s
[CV] max_features=sqrt, n_estimators=100 .............................
[CV] . max_features=sqrt, n_estimators=100, score=0.897, total=   1.0s
[CV] max_features=sqrt, n_estimators=100 .............................
[CV] . max_features=sqrt, n_estimators=100, score=0.890, total=   1.0s
[CV] max_features=sqrt, n_estimators=100 .............................
[CV] . max_features=sqrt, n_estimators=100, score=0.898, total=   1.0s
[CV] max_features=sqrt, n_estimators=100 .............................
[CV] . max_features=sqrt, n_estimators=100, score=0.885, total=   1.0s
[CV] max_features=sqrt, n_estimators=125 .............................
[CV] . max_features=sqrt, n_estimators=125, score=0.896, total=   1.3s
[CV] max_features=sqrt, n_estimators=125 .............................
[CV] . max_features=sqrt, n_estimators=125, score=0.901, total=   1.3s
[CV] max_features=sqrt, n_estimators=125 .............................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  2.5min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=99,
                               

In [26]:
print(grid.best_params_)
print(grid.best_score_)

{'max_features': 'auto', 'n_estimators': 125}
0.8954798826945327


In [27]:
# Train the model with GridSearch
grid.score(X_test_scaled, y_test)

0.9101830663615561

In [28]:
grid

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=99,
                               

In [29]:
len(X_test_scaled[0])
X_test_scaled[0].shape
X_test_scaled_point = X_test_scaled[0].reshape(1,20)
X_test_scaled_point


array([[-0.42276709, -0.57125689, -0.50369272, -0.3784998 ,  0.09043606,
         0.34473881, -0.10896433,  0.24754247, -0.26193956, -0.03430973,
        -0.84497302, -0.0462801 , -0.29592215, -0.35607115, -0.40580274,
         0.67978453, -0.16645411, -0.03351199, -1.54207582,  1.27194921]])

In [31]:
# Make a prediction!
X_test_scaled_point = X_test_scaled[0].reshape(1,20)
grid.predict(X_test_scaled_point)

array(['CONFIRMED'], dtype=object)

# Save the Model

In [32]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'alexis-random-forest.sav'
joblib.dump(grid, filename)

['alexis-random-forest.sav']